## Data Input

**Current Data Availability:**

- Monthly excess returns for **6 anomalies**.
- **Fama–French factors** (MKT, SMB, HML). _(No changes required, use as is)_.
- **Regime Classification:**
  - **Pre-Crisis:** 2003-01 to 2007-11
  - **Crisis:** 2007-12 to 2009-06
  - **Post-Crisis:** 2009-07 to 2014-05

---

## Time-Series Regression Analysis

This model evaluates whether the risk-adjusted returns of each anomaly changed significantly during the crisis regime.

### Model Specifications

**Model 1: Pre-crisis vs. Crisis**
$$R_t = \alpha + \gamma_1 \cdot \text{CrisisDummy}_t + \beta \cdot \text{Factors}_t + \epsilon_t$$

**Model 2: Crisis vs. Post-crisis**
$$R_t = \alpha + \gamma_2 \cdot \text{PostCrisisDummy}_t + \beta \cdot \text{Factors}_t + \epsilon_t$$

### Variable Definitions

- **$R_t$**: The excess return of the long–short anomaly portfolio in the month.
- **$\text{CrisisDummy}_t$**: A dummy variable equal to **1** during the crisis and **0** during the pre-crisis (Model 1).
- **$\text{PostCrisisDummy}_t$**: A dummy variable equal to **1** during the post-crisis and **0** during the crisis (Model 2).
- **$\text{Factors}_t$**: Represents the standard Fama–French three-factor model: Market (MKT), Size (SMB), and Value (HML).
- **$\gamma_1, \gamma_2$**: Coefficients capturing the change in risk-adjusted anomaly return during the crisis.
- **$\epsilon_t$**: The error term.


In [ ]:
import pandas as pd
from IPython.display import display
import statsmodels.api as sm

In [ ]:
excess_returns = pd.read_excel('./Regression Data/excess_returns.xlsx', index_col=0)
display(round(excess_returns,3))
excess_returns.info()

In [ ]:
ff_factors = pd.read_excel('./Regression Data/fama_french_factors.xlsx', index_col='date')
display(round(ff_factors.head(),3))
print(ff_factors.info())

In [ ]:
# Merge Data
# We join the anomaly returns with the Fama-French factors.
# We only need Mkt-RF, SMB, and HML from the factors df.
df_combined = pd.merge(
    excess_returns, 
    ff_factors[['Mkt-RF', 'SMB', 'HML']], 
    left_index=True, 
    right_index=True, 
    how='inner'
)
display(round(df_combined.head(),3))

In [ ]:
# List of anomaly columns (Dependent Variables)
anomalies = ['Accruals', 'Assest Growth', 'BM', 'Gross Profit', 'Momentum', 'Leaverage Ret']

In [ ]:
# ---  Define the Regression Function ---

def run_regressions(data, anomalies, dummy_col_name):
    results_list = []
    
    # Independent Variables: Constant + 3 Factors + Regime Dummy
    X_cols = ['Mkt-RF', 'SMB', 'HML', dummy_col_name]
    X = sm.add_constant(data[X_cols])
    
    for anomaly in anomalies:
        # Dependent Variable
        y = data[anomaly]
        
        # Fit OLS
        model = sm.OLS(y, X).fit(cov_type='HAC', cov_kwds={'maxlags':3})
        
        # Store results
        results_list.append({
            'Anomaly': anomaly,
            'Gamma_Coef': model.params[dummy_col_name],
            'P_Value': model.pvalues[dummy_col_name],
            'Significant (5%)': model.pvalues[dummy_col_name] < 0.05,
            't_stat': model.tvalues[dummy_col_name],
            'Beta MKT': model.params['Mkt-RF'],
            'Beta SMB': model.params['SMB'],
            'Beta HML': model.params['HML'],
            'Alpha': model.params['const'],
            'Adjusted R2': model.rsquared_adj,
        })
        
    return pd.DataFrame(results_list)


In [ ]:

# --- Run Model 1: Pre-Crisis vs. Crisis ---
# Filter: Keep only Pre-Crisis and Crisis rows
df_m1 = df_combined[df_combined['Regime'].isin(['Pre-Crisis', 'Crisis'])].copy()
print(df_m1['Regime'].unique())

# Dummy: 1 if Crisis, 0 if Pre-Crisis
df_m1['CrisisDummy'] = (df_m1['Regime'] == 'Crisis').astype(int)

results_m1 = run_regressions(df_m1, anomalies, 'CrisisDummy')
results_m1.to_excel('./Regression Results/model_1_results.xlsx', index=False)
display(round(results_m1,3))



In [ ]:
# --- Run Model 2: Crisis vs. Post-Crisis ---
# Filter: Keep only Crisis and Post-Crisis rows
df_m2 = df_combined[df_combined['Regime'].isin(['Crisis', 'Post-Crisis'])].copy()
print(df_m2['Regime'].unique())
# Dummy: 1 if Post-Crisis, 0 if Crisis
df_m2['PostCrisisDummy'] = (df_m2['Regime'] == 'Post-Crisis').astype(int)

print("\nRunning Model 2 (Crisis vs. Post-Crisis)...")
results_m2 = run_regressions(df_m2, anomalies, 'PostCrisisDummy')
results_m2.to_excel('./Regression Results/model_2_results.xlsx', index=False)
display(round(results_m2,3))